CNN Class Model Team 2 Project



In [11]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
import numpy as np

In [45]:
# Comment 0: define transformation that you wish to apply on image
data_transforms = transforms.Compose([transforms.ToTensor(),
     transforms.Resize((64,64))])

# Creating random data/images 
set = []
def random_img_maker(num_images):
    set = []
    for images in range(num_images):
        rand_img = np.random.randint(0, 256, (128, 128, 3), dtype =np.uint8)
        rand_img = data_transforms(Image.fromarray(rand_img))
        
        set.append(rand_img) 
    return set

trainset = random_img_maker(100)
testset = random_img_maker(50)

# Comment 2: Using the image datasets and the transforms, define the dataloaders
train_sampler = torch.utils.data.RandomSampler(trainset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2, sampler = train_sampler, shuffle=False, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=2, shuffle=False, num_workers=2)

classes = ('S.aureus', 'E.coli', 'B.subtilis')
print(testloader)

In [39]:
print('Example Test set: ' + str(testset[0]))
print('Example Train set: ' + str(trainset[0]))

Example Test set: tensor([[[0.4363, 0.8010, 0.7176,  ..., 0.4647, 0.5667, 0.4049],
         [0.2569, 0.7990, 0.3931,  ..., 0.6147, 0.5627, 0.2882],
         [0.4363, 0.6294, 0.6118,  ..., 0.6284, 0.5843, 0.2431],
         ...,
         [0.2951, 0.5392, 0.4343,  ..., 0.5902, 0.5304, 0.6245],
         [0.3314, 0.3157, 0.2922,  ..., 0.6775, 0.4225, 0.6216],
         [0.5843, 0.4265, 0.6843,  ..., 0.3951, 0.4020, 0.7294]],

        [[0.4657, 0.4284, 0.5029,  ..., 0.5078, 0.6382, 0.5186],
         [0.3127, 0.3873, 0.4647,  ..., 0.4892, 0.6353, 0.6118],
         [0.6971, 0.4902, 0.2588,  ..., 0.5784, 0.4725, 0.5882],
         ...,
         [0.7235, 0.3833, 0.7137,  ..., 0.3627, 0.5225, 0.5343],
         [0.5216, 0.6314, 0.7235,  ..., 0.4020, 0.4324, 0.4804],
         [0.5794, 0.4539, 0.5294,  ..., 0.5961, 0.3931, 0.6647]],

        [[0.5716, 0.6520, 0.3647,  ..., 0.4314, 0.6363, 0.5892],
         [0.4441, 0.3196, 0.2275,  ..., 0.3471, 0.8343, 0.6373],
         [0.4647, 0.5324, 0.3853,  ..., 

CNN Model

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1)

        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1)

        self.convert = nn.Conv2d(in_channels = 3, out_channels= 32, kernel_size = 1, stride = 2)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(32 * 16 * 16, 128)                                
        self.fc2 = nn.Linear(128, 64)

        self.fc3 = nn.Linear(64, 6)


    def forward(self, x):
        # Save the input for the skip connection
        residual = x
        #print(self.convert(residual).shape)

        x = self.conv1(x)    # initial layer

        x = self.relu(x)

        x = self.pool(x)

        x = self.conv2(x)   # second conv

        x = (x) + self.convert(residual)

        x = self.relu(x)

        x = self.pool(x)

        x = x.view(-1, 32 * 16 * 16)

        # Fully connected layers
        x = self.fc1(x)

        x = self.fc2(x)

        x = self.fc3(x)

        return x

net = Net()

# Instantiate the network
test_net = Net()

output = test_net(trainset[0])

# Print the output
print(output)


tensor([[-0.0130,  0.2555,  0.0379, -0.2074, -0.0712,  0.1499]],
       grad_fn=<AddmmBackward0>)


In [41]:
criterion =  nn.CrossEntropyLoss()
sgd = torch.optim.SGD(net.parameters(), lr = 0.005, momentum = 0.9)

# load net to device
device = torch.device('cuda')

In [42]:
def train_random_sample(net, optimizer, device, trainloader):

    net.train()
    net.to(device)
    
    for epoch in range(10): 

        running_loss = 0.0
        for i, data in enumerate(trainloader):

            first = data[0].to(device)
            truth = data[1].to(device)

            optimizer.zero_grad()
            forward_pass = net(first)

            # loss
            loss = criterion(forward_pass, truth)
            # back prop
            loss.backward()
            # optimze with the descent
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 25 == 24:  # print every 5 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 25))
                running_loss = 0.0

    print('Finished Training')
    return net


In [44]:
# kick off the training
net = train_random_sample(net, sgd, device, trainloader)
